## A MultiServer Queue

Let's develop a simulation for single FIFO Queue that leads to 3 servers.  
 - Customer arrivals follow a poisson process with rate $\lambda$, (i.e. the interarrival times are exponentially distributed with mean $1/\lambda$).
 - Service Times are follow an exponential distribution with mean $1/\mu$.
 - There are $c$ different servers.

 We want to simulate one replication path with $numCustomer$ arrivals.

In [12]:
import numpy as np  

Write a function that takes in $\lambda$, $\mu$, $c$, numCustomers, simulates one replication path, and returns ther arrays A, S, D of the arrival times, start of service, and departure times for the first numCustomer arrivals.

#### For you:  Write a function that takes in the arrays A, S, D and computes 
 - The average waiting time across customers
 - The average length of the queue (averaged over time)

### (For you): Compute the expected average waiting time across the first 100 arrivals using a simulation with 1000 path

### More interesting service times
Suppose you believe that 10\% of customers have very difficult requests that take a long time for service, say they take $1/\mu_2 > 1/\mu_1$ time.  How would you alter your simulation?